# BQ dbt mart source data extracts
Creates the simulated source data extract files in a BigQuery storage bucket.

Data extracts are based on the GCP public data set `bigquery-public-data.thelook_ecommerce` for a defined controlled
time window - e.g. 14 days.  

A week is taken to be Mon - Sun to mimic the 'start-of-business' Monday scenario at the end of the defined time window.

Included data sets:
  - `bigquery-public-data.thelook_ecommerce.users`

---
FYI - The base SQL extract statements were initally created using the dbt analysis SQL in `./bq-dbt-mart/sample_bq_dbt_mart/analyses`.

The SQL statements are rendered in the `<dbt-project-dir>/target/compiled/analysis/...` directory.

In [42]:
# project imports
from google.cloud import bigquery
import pendulum
from structlog import get_logger


In [7]:
# load required notebook extensions
%load_ext google.cloud.bigquery


The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [43]:
# setup the BigQuery client connection
bq_client = bigquery.Client()

# setup a formatted logger
logger = get_logger()

In [44]:
# batch run parameters
batch_start_dt = pendulum.parse('2023-12-04').date()
batch_end_dt = batch_start_dt.add(days=13)

raw_data_bucket = 'data-pipeline-tools-raw-data'

logger.info('Batch interval', batch_start_dt=batch_start_dt.to_date_string())
logger.info('Batch interval', batch_end_dt=batch_end_dt.to_date_string())

2024-01-04 15:20:50 [info     ] Batch interval                 batch_start_dt=2023-12-04
2024-01-04 15:20:50 [info     ] Batch interval                 batch_end_dt=2023-12-17


In [58]:
def full_refresh_extract_sql(src_table: str, batch_dt: pendulum.date, extract_bucket: str, extract_prefix: str) -> str:

    partition_key = f"p_year={batch_dt.year:04d}/p_month={batch_dt.month:02d}/p_day={batch_dt.day:02d}"

    file_dttm_str = batch_dt.format('YYYY-MM-DD') + 'T000000-0'

    sql = f"""
    export data options (
        uri='gs://{ extract_bucket }/{ extract_prefix }/full-refresh/{ src_table }/{ partition_key }/*.csv',
        format='CSV',
        overwrite=true,
        header=true
    ) AS
    SELECT *
    FROM `bigquery-public-data`.`thelook_ecommerce`.`{ src_table }`
    WHERE
        created_at < CAST('{ batch_dt.to_date_string() } 00:00:00' AS TIMESTAMP FORMAT 'YYYY-MM-DD HH24:MI:SS' AT TIME ZONE 'Australia/Melbourne') + INTERVAL '1' DAY;
    """
    return sql


In [59]:
source_table = 'users'
target_source_name = 'thelook'

for dt in pendulum.interval(batch_start_dt, batch_end_dt).range('days'):
    logger.info('batch date', dt=dt.to_date_string())

    extract_sql = full_refresh_extract_sql(source_table, dt, raw_data_bucket, target_source_name)
    logger.debug('extract SQL', sql=extract_sql)
    query_job = bq_client.query(extract_sql)
    results = query_job.result()

    logger.info('result result', status=query_job.state, errors=query_job.errors)



2024-01-04 15:51:54 [info     ] batch date                     dt=2023-12-04
2024-01-04 15:51:54 [debug    ] extract SQL                    sql=
    export data options (
        uri='gs://data-pipeline-tools-raw-data/thelook/full-refresh/users/p_year=2023/p_month=12/p_day=04/*.csv',
        format='CSV',
        overwrite=true,
        header=true
    ) AS
    SELECT *
    FROM `bigquery-public-data`.`thelook_ecommerce`.`users`
    WHERE
        created_at < CAST('2023-12-04 00:00:00' AS TIMESTAMP FORMAT 'YYYY-MM-DD HH24:MI:SS' AT TIME ZONE 'Australia/Melbourne') + INTERVAL '1' DAY;
    
2024-01-04 15:52:02 [info     ] result result                  errors=None status=DONE
2024-01-04 15:52:02 [info     ] batch date                     dt=2023-12-05
2024-01-04 15:52:02 [debug    ] extract SQL                    sql=
    export data options (
        uri='gs://data-pipeline-tools-raw-data/thelook/full-refresh/users/p_year=2023/p_month=12/p_day=05/*.csv',
        format='CSV',
        ov